# Investigation of coalesence for LOW data

Baseline dependent averaging is a form of data coalescence. In this script, we create a critically sampled snapshot of a LOW data set, and then coalesce and decoalesce it to see what errors result. We look at the time required for all steps.

In [ ]:
% matplotlib inline

import os
import sys

sys.path.append(os.path.join('..', '..'))

from matplotlib import pylab

pylab.rcParams['agg.path.chunksize'] = 10000

from libs.data.parameters import arl_path
results_dir = arl_path('test_results')



import numpy

from astropy.convolution import Gaussian2DKernel

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt
from matplotlib.pyplot import cm

from libs.data.polarisation import PolarisationFrame
from libs.visibility.base import create_blockvisibility, copy_visibility
from libs.skycomponent.operations import create_skycomponent, insert_skycomponent, apply_beam_to_skycomponent
from libs.image.operations import show_image, smooth_image
from libs.imaging.base import create_image_from_visibility
from libs.fourier_transforms.fft_support import extract_mid
from libs.visibility.coalesce import coalesce_visibility
from libs.image.iterators import image_raster_iter
from libs.visibility.iterators import vis_timeslice_iter
from libs.util.testing_support import create_named_configuration, create_low_test_beam, \
    create_low_test_skycomponents_from_gleam
from libs.imaging import *

import logging

def init_logging():
    log = logging.getLogger()
    logging.basicConfig(filename='%s/imaging-coalesce.log' % results_dir,
                        filemode='a',
                        format='%(thread)s %(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.INFO)
log = logging.getLogger()
logging.info("Starting imaging-coalesce")


In [ ]:
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'Greys'

Construct the configuration and fill in the appropriate sampling values

In [ ]:
config = 'core'
if config == 'full':
    b = 8e4
    low = create_named_configuration('LOWBD2', rmax=b)
    cellsize = 0.00001
    npixel=8192
    nsnapshots = 30

else:
    b = 1e3
    low = create_named_configuration('LOWBD2', rmax=b)
    cellsize = 0.0003
    npixel=1024
    nsnapshots = 30

    
oversampling = 2

sampling_time = 35.0 / (oversampling * b)
log.info("Critical sampling time = %.5f (radians) %.2f (seconds)" % 
         (sampling_time, sampling_time * 43200.0 / numpy.pi))
sampling_frequency = 1e8 * 35.0 / (oversampling * b) 
log.info("Critical sampling frequency = %.5f (Hz) " % (sampling_frequency))
times = numpy.arange(0.0, + nsnapshots * sampling_time, sampling_time)
frequency = numpy.linspace(1e8 - sampling_frequency, 1e8 + sampling_frequency, 3)
channel_bandwidth = numpy.full_like(frequency, sampling_frequency)

log.info("Observing frequencies %s Hz" % (frequency))

log.info("Cellsize = %.6f radians" % (cellsize))

We create the visibility holding the vis, uvw, time, antenna1, antenna2, weight columns in a table. The actual visibility values are zero.

In [ ]:
phasecentre = SkyCoord(ra=+355.0 * u.deg, dec=-10.0 * u.deg, frame='icrs', equinox='J2000')
vt = create_blockvisibility(low, times, frequency, channel_bandwidth=channel_bandwidth,
                       weight=1.0, phasecentre=phasecentre, polarisation_frame=PolarisationFrame('stokesI'))

Create components from GLEAM 

In [ ]:
comps = create_low_test_skycomponents_from_gleam(flux_limit=1.0, polarisation_frame=PolarisationFrame("stokesI"),
                                             frequency=frequency, phasecentre=phasecentre, 
                                                 radius=0.2)

In [ ]:
model = create_image_from_visibility(vt, npixel=npixel, cellsize=cellsize, frequency=frequency,
                                     polarisation_frame=PolarisationFrame('stokesI'),
                                     phasecentre=phasecentre, nchan=len(frequency))
beam=create_low_test_beam(model)
comps = apply_beam_to_skycomponent(comps, beam)
model = insert_skycomponent(model, comps)

In [ ]:
show_image(beam, cm='Greys', components=comps, vmax=1.0, vmin=0.0)
plt.title("Beam")
plt.show()

In [ ]:
cmodel = smooth_image(model)
show_image(cmodel, vmax=1, vmin=0.0, cm='Greys')
plt.title("Smoothed model image")
plt.show()

Since we are using a BlockVisibility, we can only predict the visibility from sky components. 

In [ ]:
vt = predict_skycomponent_visibility(vt, comps)

Now we coalesce the data

In [ ]:
time_coal=1.0
max_time_coal=100
frequency_coal=0.0
max_frequency_coal=1

cvt = coalesce_visibility(vt, time_coal=time_coal, frequency_coal=frequency_coal)
plt.clf()
plt.plot(+cvt.uvw[:,0],+cvt.uvw[:,1], '.', color='b')
plt.plot(-cvt.uvw[:,0],-cvt.uvw[:,1], '.', color='b')
plt.title('Coalesced uv coverage')
plt.xlabel('U (lambda)')
plt.ylabel('V (lambda)')
plt.show()

Check that we did not coalesce in frequency and that we did in time

In [ ]:
numpy.unique(cvt.frequency)==vt.frequency

In [ ]:
numpy.unique(cvt.time).size > vt.time.size

Make the dirty image from coalesced data

In [ ]:
from libs.imaging import advise_wide_field
advise_wide_field(cvt)
dirtyimage, sumwt = invert_2d(cvt, model)

In [ ]:
show_image(dirtyimage, vmax=1.0, vmin=-0.1, cm='Greys')
plt.title('Coalesced data image')
plt.show()

In [ ]:
cvtpred = copy_visibility(cvt, zero=True)
cvtpred = predict_skycomponent_visibility(cvtpred, comps)

In [ ]:
plt.clf()
plt.plot(vt.vis.real.flatten(), vt.vis.imag.flatten(), '.', color='g', label='BlockVisibility')
plt.xlabel('Real')
plt.ylabel('Imaginary')
plt.title('Original visibility')
plt.legend()
plt.show()

plt.clf()
plt.plot(cvt.vis.real.flatten()-cvtpred.vis.real.flatten(), 
         cvt.vis.imag.flatten()-cvtpred.vis.imag.flatten(), '.', color='r', label='Error')
plt.title('Error between coalesce/predict and predict/coalesce')
plt.xlabel('Real')
plt.ylabel('Imaginary')
plt.legend()
plt.show()